In [1]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime
from utils import *
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:

# Download NLTK model data (you need to do this once)# Downl 
# nltk.download("book")

In [3]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Read the data and append SENTENCE_START and SENTENCE_END tokens
print "Reading CSV file..."
with open('textfile.txt', 'rb') as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.next()
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print "Parsed %d sentences." % (len(sentences))
    
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_freq.items())

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print "Using vocabulary size %d." % vocabulary_size
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print "\nExample sentence: '%s'" % sentences[0]
print "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0]

Reading CSV file...
Parsed 79170 sentences.
Found 65440 unique words tokens.
Using vocabulary size 8000.
The least frequent word in our vocabulary is 'fortunately' and appeared 10 times.

Example sentence: 'SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END'

Example sentence after Pre-processing: '[u'SENTENCE_START', u'i', u'joined', u'a', u'new', u'league', u'this', u'year', u'and', u'they', u'have', u'different', u'scoring', u'rules', u'than', u'i', u"'m", u'used', u'to', u'.', u'SENTENCE_END']'


In [4]:
# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])


In [5]:
X_train1 = []
for element in X_train:
    X_train2 = []
    for element1 in element:
        X_train2.append([element1])
    X_train1.append(X_train2)

In [13]:
for i in X_train1:
    print(len(i))

20
10
28
18
24
15
27
9
25
16
45
15
8
32
7
3
7
10
11
19
21
28
17
13
17
6
3
16
9
11
25
35
12
5
7
12
20
12
11
14
25
10
19
19
32
21
30
12
38
25
19
24
14
7
15
6
30
10
14
22
44
11
8
14
18
9
5
72
32
38
8
43
54
7
25
63
12
16
8
13
7
13
9
14
33
23
13
17
41
33
36
63
13
14
27
10
15
14
24
23
18
12
12
7
11
19
10
9
6
7
25
25
43
8
17
21
28
3
21
21
5
6
16
11
19
6
5
9
30
27
19
13
37
7
10
30
17
15
14
16
26
12
5
30
28
9
11
15
30
16
7
7
15
35
15
26
19
29
22
10
7
11
5
28
35
16
23
16
31
13
29
33
32
26
4
19
27
22
7
17
26
20
20
23
45
35
16
37
13
29
15
21
23
2
30
14
5
49
19
38
6
18
31
9
26
18
14
20
20
13
30
11
28
17
20
14
8
23
21
12
25
29
26
7
31
19
18
12
10
31
44
31
29
46
23
15
23
22
3
18
19
23
18
25
33
38
13
16
12
16
7
39
14
7
26
34
12
7
13
5
15
18
16
8
16
15
19
12
31
43
30
13
10
41
55
18
17
7
33
18
11
30
12
20
38
24
21
30
33
25
48
44
32
37
26
36
128
55
52
2
8
21
19
18
23
27
15
8
15
11
11
11
21
12
47
12
26
50
3
28
23
24
14
14
14
8
14
14
17
8
26
12
28
25
16
16
12
13
17
14
22
12
22
17
29
14
13
22
24
11
6
11
49


12
23
49
11
29
12
14
6
11
16
60
23
16
11
20
18
22
17
2
23
9
20
8
16
27
25
14
6
27
6
15
49
21
9
12
25
5
13
17
33
15
16
26
31
18
31
39
36
45
12
22
14
25
21
14
53
33
27
9
18
9
18
23
11
12
15
37
29
29
15
38
4
20
20
10
27
46
17
11
6
4
15
20
24
25
34
50
38
30
21
25
69
23
44
15
12
9
30
18
10
45
20
9
12
15
27
13
4
29
17
25
30
24
24
14
9
12
12
13
8
19
17
11
19
9
21
55
31
31
16
18
17
19
23
2
29
49
23
3
12
34
11
35
39
17
45
24
10
9
8
43
18
6
21
23
26
18
8
18
9
27
23
18
23
13
11
28
30
14
47
33
13
32
49
164
44
22
18
9
21
41
27
19
19
26
4
25
47
61
20
42
20
33
11
35
9
49
7
36
57
8
26
28
20
9
106
36
9
7
19
25
14
27
11
8
7
19
32
21
12
32
14
30
36
24
17
11
14
12
10
11
8
26
35
26
34
34
22
15
24
34
32
18
9
14
16
7
17
25
20
26
20
21
23
4
33
34
13
24
19
36
14
21
45
28
17
8
18
44
6
13
31
24
57
19
27
5
27
18
19
25
8
7
11
24
20
35
21
11
10
8
15
24
9
27
15
11
25
21
22
17
37
8
16
28
25
26
22
44
4
31
11
12
22
18
21
12
25
18
16
21
31
16
18
17
19
23
2
19
23
15
10
14
19
26
31
5
30
11
34
39
22
23
36
25
12
37
33
38
26

26
26
17
46
5
21
13
23
43
26
25
11
21
16
29
15
27
18
20
26
24
26
22
39
21
35
28
19
38
16
33
15
26
10
10
15
16
12
55
43
14
7
26
26
5
19
42
20
5
42
35
27
12
13
23
36
10
50
11
18
19
25
17
21
13
6
3
9
9
8
12
17
5
15
10
22
20
3
19
16
9
28
31
35
17
20
18
19
18
17
19
13
23
2
30
17
5
6
9
20
24
20
13
20
13
9
24
6
9
13
15
26
20
10
15
12
43
9
23
14
31
13
31
3
8
29
55
8
24
17
34
19
17
6
7
34
19
8
19
20
9
29
9
16
10
8
9
15
12
7
15
41
26
26
26
22
12
18
30
17
25
53
23
8
20
16
37
22
19
22
18
27
27
9
26
5
9
11
5
10
10
8
8
9
13
7
15
7
22
14
8
17
8
11
21
24
13
12
52
10
5
16
30
28
34
24
25
9
4
25
17
24
12
9
12
15
10
15
16
6
9
17
29
28
8
21
12
14
10
14
23
16
26
32
49
16
28
28
19
21
18
11
17
19
15
12
22
14
23
26
7
71
18
49
19
21
28
11
25
24
6
12
31
9
8
9
70
41
78
40
17
26
26
23
23
35
14
24
13
20
25
18
15
8
2
16
28
8
13
8
13
8
13
8
35
8
20
9
7
8
3
25
13
23
5
11
15
39
33
32
10
3
15
17
27
21
21
25
5
20
14
72
53
25
24
16
19
26
40
27
47
73
34
18
14
21
19
7
25
23
23
20
11
16
21
7
11
19
13
5
10
27
13
15
26
5
15
16

45
27
13
14
25
21
11
28
21
9
16
18
11
12
48
69
7
18
42
19
8
9
6
7
52
34
39
3
16
14
16
5
22
36
45
28
22
25
6
8
15
23
4
51
4
24
15
17
17
45
6
49
57
57
21
36
26
12
17
14
25
3
30
39
5
23
19
9
8
5
13
15
19
28
6
15
29
16
7
30
42
23
11
43
4
12
11
6
18
6
33
33
28
7
17
9
44
9
4
35
12
24
25
18
19
34
5
10
4
27
76
4
14
10
28
14
19
8
10
22
26
30
9
11
15
23
24
22
9
16
32
23
20
17
5
11
24
18
27
14
16
21
26
22
23
25
13
19
21
14
18
42
18
10
11
13
28
29
26
37
11
11
25
25
30
68
29
10
33
35
22
39
12
14
10
2
2
53
27
21
7
14
40
13
17
13
21
24
29
12
8
48
15
14
15
27
16
15
21
23
2
40
13
42
61
38
10
33
7
5
48
17
30
24
23
41
21
21
16
11
5
17
24
19
23
26
8
19
13
13
52
13
32
14
18
27
13
12
10
24
16
16
46
55
25
27
19
43
15
21
11
9
23
34
27
27
21
16
19
65
9
20
33
25
48
24
13
23
2
19
26
19
13
23
2
5
5
15
17
21
27
26
12
18
9
8
10
23
9
16
18
14
14
16
9
22
16
61
19
16
39
23
27
19
68
58
19
24
17
25
26
24
4
22
17
31
10
22
9
18
9
31
10
18
32
24
8
44
18
28
32
35
4
32
7
4
5
11
22
14
8
40
15
22
21
17
13
48
30
31
27
21
21
25


17
5
46
21
29
25
38
81
22
14
50
25
25
8
20
23
17
25
32
13
7
26
29
45
27
10
43
3
5
19
8
30
25
14
5
7
20
26
20
13
18
29
14
38
32
13
26
17
3
4
12
21
10
13
19
18
17
15
10
23
11
23
24
20
18
18
11
4
46
34
32
21
25
34
8
13
2
2
27
26
4
17
20
25
52
13
26
42
7
40
16
22
36
18
5
11
21
21
28
22
5
34
28
25
56
17
22
23
31
39
44
19
21
20
20
8
28
39
24
21
33
38
4
11
11
35
24
51
59
54
8
59
29
15
11
20
35
20
10
25
19
7
29
23
2
11
12
6
41
4
49
45
14
32
8
36
11
16
24
13
19
22
15
26
26
3
59
29
24
20
19
10
13
23
2
26
20
14
7
12
29
31
17
27
7
30
15
32
20
12
7
14
6
25
30
27
7
20
19
18
15
15
14
12
3
9
3
3
13
24
22
39
27
11
23
6
18
61
26
40
22
26
30
22
22
23
10
11
8
8
18
20
9
6
17
8
5
5
13
12
9
15
14
7
25
14
17
17
31
16
19
22
27
16
31
16
18
17
19
23
2
33
14
26
20
10
15
45
66
47
6
16
12
12
16
24
28
52
18
24
19
15
23
56
24
27
31
27
9
9
13
38
20
15
10
17
17
7
4
9
9
20
21
30
26
14
27
18
16
13
25
14
18
16
4
23
22
20
7
12
46
4
58
21
10
38
14
14
11
18
3
14
15
26
6
16
34
10
26
23
31
10
8
11
21
12
8
8
13
55
34
13
46
17
1

...

17
36
40
17
4
5
13
25
97
19
33
12
35
30
11
39
23
25
50
27
8
25
15
24
16
25
23
18
19
8
30
32
6
23
25
8
23
33
36
29
14
39
12
14
20
18
14
15
16
15
36
28
10
10
23
26
27
12
19
12
46
19
37
31
11
18
11
8
9
31
44
4
34
8
22
22
11
21
14
30
8
13
9
39
5
28
33
24
19
15
9
7
62
14
10
23
8
21
11
45
30
33
17
15
15
22
6
10
10
35
35
5
8
3
11
33
5
20
15
19
19
47
17
4
46
29
11
46
34
13
16
23
2
4
6
67
7
54
8
6
19
19
21
6
13
37
3
4
9
19
12
20
16
22
18
12
6
6
12
20
22
16
28
18
11
43
16
16
25
30
23
28
16
35
23
10
11
8
27
31
26
9
21
19
17
20
10
14
22
30
23
25
9
18
51
15
7
10
29
3
12
2
32
21
34
21
33
33
28
19
13
8
34
13
13
9
20
13
11
16
15
22
10
14
17
36
30
15
14
20
12
41
9
10
20
21
15
16
12
18
5
10
23
8
31
25
44
16
10
16
14
64
12
21
7
8
21
36
40
13
8
10
10
9
29
8
14
9
40
29
26
33
25
5
33
25
18
10
13
41
23
14
14
34
37
22
36
10
24
3
48
23
3
39
77
3
22
19
31
18
13
26
24
11
17
4
3
16
59
23
12
18
32
29
44
17
14
5
31
20
26
29
13
33
16
18
21
7
19
13
18
50
11
5
5
7
6
4
17
19
17
27
18
16
26
37
26
20
6
13
23
2
20
11
22
8

21
23
9
6
12
20
13
12
23
12
8
8
20
19
15
11
14
18
30
31
20
17
39
32
21
13
41
21
17
2
16
27
38
29
17
38
14
14
9
39
8
33
11
7
10
18
24
46
5
48
22
21
37
38
38
26
26
35
3
22
23
35
36
5
4
25
20
10
9
10
6
5
8
24
28
10
25
32
24
37
21
53
16
36
9
24
15
13
24
14
20
17
13
12
12
21
2
14
32
11
14
13
17
14
23
37
26
27
9
27
22
34
23
5
33
17
26
31
25
22
77
11
17
18
36
18
15
17
8
12
18
13
15
3
3
10
18
7
9
4
8
7
12
15
17
44
6
26
31
17
14
16
43
12
11
9
9
11
10
17
9
17
60
47
14
10
35
10
18
18
21
5
15
12
13
12
11
29
11
3
26
21
9
13
13
11
25
13
30
135
32
24
10
30
17
16
14
36
47
23
2
5
33
36
6
13
10
19
6
6
30
15
8
29
39
17
14
23
16
10
22
18
10
15
21
34
27
33
27
29
39
11
32
24
12
15
15
13
8
6
20
13
27
14
24
25
30
19
9
11
21
13
15
3
6
13
39
3
7
8
47
10
18
15
18
29
13
28
26
14
16
12
26
14
17
12
12
32
34
9
33
17
13
11
27
29
62
34
33
15
17
10
16
36
45
33
13
24
17
29
18
5
8
19
8
6
3
17
11
48
5
22
22
8
21
16
10
11
9
13
11
24
13
21
6
18
13
14
13
5
8
18
9
15
14
5
7
35
12
25
19
25
28
36
30
35
18
29
19
8
19
19
14
32
17

10
7
10
53
29
38
20
23
33
21
47
47
25
15
14
12
14
13
32
28
30
63
9
22
15
21
29
14
32
45
15
38
20
9
11
15
27
44
14
7
4
30
11
29
8
10
20
15
19
6
5
8
9
3
5
6
12
5
11
13
10
18
19
17
14
23
2
3
49
11
15
7
5
29
16
12
17
10
12
17
17
26
3
49
9
3
45
19
29
14
27
10
4
9
7
9
8
3
10
11
6
7
11
9
19
30
35
31
19
17
11
66
42
10
14
41
47
41
27
5
20
13
25
7
5
14
25
8
9
7
37
23
20
34
12
25
21
22
14
19
13
32
15
20
5
14
15
44
27
28
5
35
4
9
41
3
11
39
35
13
24
5
9
19
9
7
23
7
14
18
35
20
21
18
14
18
27
15
11
35
34
10
43
9
4
26
13
13
27
24
40
30
31
69
34
5
12
18
18
31
65
27
16
10
14
21
4
11
29
18
7
5
6
25
21
19
7
32
22
24
26
23
7
7
15
29
16
10
7
31
38
5
15
17
21
9
21
15
14
19
4
13
14
25
15
24
22
14
39
50
4
9
42
17
66
25
35
28
34
29
46
27
16
31
8
16
37
7
8
16
16
8
7
14
26
6
9
22
7
21
29
12
13
15
27
7
33
9
37
15
23
15
25
21
33
16
31
8
2
37
18
10
9
4
7
24
12
20
20
19
54
16
22
38
38
34
23
43
5
22
35
6
45
25
42
19
23
22
24
46
13
36
28
12
18
28
31
22
14
5
59
7
36
13
40
10
24
42
49
33
21
33
18
5
24
30
24
11
15
33
25

27
29
18
32
14
16
7
20
11
33
44
14
24
18
26
17
14
26
12
14
13
22
10
29
13
20
21
3
27
24
56
51
33
19
6
29
30
21
8
15
10
15
23
13
12
17
20
26
21
75
27
47
89
6
8
21
14
14
12
9
7
9
15
14
8
10
21
20
24
39
14
31
16
18
17
19
23
2
11
4
32
25
17
13
6
15
25
27
37
15
76
27
35
15
26
24
24
13
47
9
43
24
4
43
46
29
11
8
6
37
22
31
8
21
20
18
26
8
7
45
42
19
24
14
23
37
46
21
20
8
54
36
20
30
39
35
14
14
24
9
9
11
3
8
5
6
4
78
16
11
8
24
18
36
26
22
16
8
4
4
12
8
4
14
6
9
4
8
12
14
6
12
5
8
6
12
15
6
10
5
9
37
16
20
25
16
19
8
40
17
25
12
24
19
21
45
28
13
12
15
21
9
19
10
15
10
9
9
10
15
3
10
18
11
9
20
21
17
15
2
16
14
29
7
21
22
22
21
18
25
27
14
8
10
33
24
7
23
34
27
13
23
2
23
34
27
13
23
2
51
56
14
22
11
33
30
17
19
21
14
10
20
3
5
12
21
18
28
4
26
40
15
3
15
8
12
8
15
20
17
3
8
23
22
28
5
38
9
33
25
17
6
43
21
45
16
14
15
22
35
16
29
12
10
5
4
6
12
18
28
13
10
19
5
20
32
7
25
10
14
20
6
5
19
10
16
18
46
20
68
4
7
10
24
43
11
13
18
17
23
19
15
14
19
21
16
24
16
14
21
13
25
22
8
24
25
20
10
8
35

15
14
7
15
35
28
37
17
18
19
72
29
39
13
10
27
24
40
27
11
6
10
11
13
14
18
15
3
12
32
15
20
266
12
11
11
5
12
6
20
13
12
32
22
19
27
14
10
14
14
8
32
33
28
12
22
3
10
20
23
16
9
27
14
22
16
29
6
32
36
21
33
6
40
76
34
19
43
28
39
44
8
14
10
53
18
16
25
12
22
32
5
29
20
17
50
7
16
19
13
19
27
16
44
23
17
17
23
37
25
15
5
12
10
15
32
13
6
33
51
18
40
4
39
16
4
7
8
23
18
8
26
45
22
6
36
15
20
14
36
19
46
15
8
19
8
11
23
10
27
24
46
19
14
24
5
12
14
15
3
15
23
2
44
11
12
12
35
12
14
12
15
19
12
20
17
8
4
24
42
30
25
24
6
4
7
34
16
24
7
20
33
24
16
31


In [7]:
pickle.dump(X_train1[:10000],open("train_text",'wb'))
pickle.dump(X_train1[10000:20000],open("test_text",'wb'))

In [ ]:
check = 10
for i in X_train:
    if max(i) < check:
        check = max(i)

In [ ]:
check